# Estruturas Cripográficas - Criptografia e Segurança da Informação

## TP2 - Exercício 1

#### Enunciado

Estes problemas destinam à iniciação do uso do SageMath  em protótipos de esquemas clássicos de chave pública.

1. Construir uma classe Python que implemente o  EdDSA a partir do [“standard” FIPS186-5](https://csrc.nist.gov/publications/detail/fips/186/5/draft)
    1. A implementação deve conter funções para assinar digitalmente e verificar a assinatura.
    2. A implementação da classe deve usar  uma das “Twisted Edwards Curves” definidas no standard e escolhida  na iniciação da classe: a curva  “edwards25519” ou “edwards448”.

# Versão do SageMath 

`SageMath 9.5`

 (Confirmar não se vou mudar)

METER PASSOS PARA INSTALAR + SECCÇÃO COM IMPORTS EM BAIXO


In [39]:
#import sage
import os
import hashlib

#### Classe EdDSA - Capítulo 7 do FIPS186-5

No capitulo 7 tem de ter o input

Classe fornecida para representar pontos *edward*

In [40]:
class EdPnt(object):
    def __init__(self,pt=None,curve=None,x=None,y=None):
        if pt != None:
            self.curve = pt.curve
            self.x = pt.x
            self.y = pt.y
            self.w = pt.w
        else:
            self.curve = curve
            self.x = x
            self.y = y
            self.w = x * y
    
    def eq(self, other):
        return self.x == other.x and self.y == other.y
    
    def copy(self):
        return EdPnt(curve=self.curve, x=self.x, y=self.y)
    
    def zero(self):
        return EdPnt(curve=self.curve ,x=0, y=1)
    
    def sim(self):
        return EdPnt(curve=self.curve, x= -self.x, y= self.y)
    
    def soma(self, other):
        a = self.curve.a
        d = self.curve.d
        delta = d * self.w * other.w

        self.x,self.y = (self.x * other.y + self.y * other.x)/(1 + delta),(self.y * other.y - a * self.x * other.x)/(1 - delta)
        self.w = self.x * self.y
        
    def duplica(self):
        a = self.curve.a
        d = self.curve.d
        delta = d*(self.w)**2

        self.x,self.y = (2 * self.w)/(1 + delta),(self.y**2 - a * self.x**2)/(1 - delta)
        self.w = self.x * self.y
        
    def mult(self, n):
        m = Mod(n, self.curve.L).lift().digits(2)
        Q = self.copy()
        A = self.zero()

        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()

        return A

funções auxiliares

In [41]:
def sha512(x):
    return hashlib.sha512(str(x).encode()).digest()

def sha512_modq(s): 
    return int.from_bytes(sha512(s), "little") % ZZ(2**252 + 27742317777372353535851937790883648493)

def modp_inv(x, p): 
    return pow(x, p-2, p)

def recover_x(y, sign, p, d):
    if y >= p: 
        return None
    
    x2 = (y*y-1) * modp_inv(d*y*y+1, p)

    if x2 == 0:
        if sign:
             return None
        else: 
            return 0
        
    x = pow(x2, (p+3) // 8, p)
    x = int(x.lift())
    
    if (x*x - x2) % p != 0:
        x = x * pow(2, (p-1) // 4, p) % p 

    if (x*x - x2) % p != 0:
        return None
    
    if (x & 1) != sign:
        x = p - x

    return x

def point_decompress(s,p,d):
    if len(s) != 32:
        raise Exception("Invalid input length for decompression") 
    
    y = int.from_bytes(s, "little")
    sign = y >> 255
    y &= (1 << 255) - 1

    x = recover_x(y, sign, p, d)

    if x is None:
         return None
    else:
        return (x, y, 1, x*y % p)

def point_compress(P, p):
    p_x = P.x
    p_y = P.y
    p_z = 1

    zinv = modp_inv(p_z,p)
    x = p_x * zinv % p
    y = p_y * zinv % p

    return int.to_bytes(int(int(p_y) | ((int(p_x) & 1) << 255)), 32, "little")

In [42]:
class EdCDSA:
    def __init__(self,p,b,n,K,a,d,Px,Py,L,c):
        self.a = a
        self.d = d
        self.p = p
        self.K = K
        self.L = L
        
        self.Px = Px
        self.Py = Py

        assert a != d and is_prime(p) and p > 3
        
        A = 2 * (a + d)/(a - d)
        B = 4/(a - d)
        self.alfa = A/(3*B)
        self.s = B

        a4 = self.s**(-2) - 3*self.alfa**2
        a6 = (-self.alfa)**3 - a4*self.alfa
 
        self.EC = EllipticCurve(K, [a4, a6])
        self.P = self.ed2ec(Px,Py)

    #Calculate private key
    def private_key(self, seed):
        if (len(seed) != 32):
            raise Exception("Bad seed length")
        
        hashed_seed = sha512(seed)
        a = int.from_bytes(hashed_seed[:32], "little")
        a &= (1 << 254) - 8
        a |= (1 << 254)

        return (a, hashed_seed[32:])
        
    #Calculate public key
    def public_key(self, seed):
        (a,dummy) = self.private_key(seed)

        point = EdPnt(curve=self, x=self.Px, y=self.Py)
        point = point.mult(a)

        return point_compress(point, self.p)

    #Sign message
    def sign(self, message, seed):
        (a,prefix) = self.private_key(seed) 
        #Public key calculation
        point_A = EdPnt(curve=self, x=self.Px, y=self.Py)
        point_A = point_A.mult(a)
        A = point_compress(point_A, self.p)
        #Secret number calculation
        r = sha512_modq(prefix + message)

        #Ponto R
        point_R = EdPnt(curve=self, x=self.Px, y=self.Py)
        point_R = point_R.mult(r)
        Rs = point_compress(point_R, self.p)

        h = sha512_modq(Rs + A + message)
        s = (r + h * a) % self.L

        return Rs + int.to_bytes(int(s), 32, "little")

    #Verify message signature
    def verify(self, message, signature, public_key):
        if len(public_key) != 32: 
            raise Exception("Bad public key length") 
        if len(signature) != 64: 
            Exception("Bad signature length") 
        
        #Obter ponto representativo da public key
        A = point_decompress(public_key, self.p, self.d)
        if not A: 
            return False 
        
        #Obter ponto r
        Rs = signature[:32]
        R = point_decompress(Rs, self.p, self.d) 
        point_r = EdPnt(curve=self, x=R[0], y=R[1])

        if not R:
            return False 
        
        s = int.from_bytes(signature[32:], "little") 
        if s >= self.L: 
            return False 
        
        h = sha512_modq(Rs + public_key + message) 

        point_s = EdPnt(curve=self, x=self.Px, y=self.Py)
        point_s = point_s.mult(s)
        
        point_h = EdPnt(curve=self, x=A[0], y=A[1])
        point_h = point_h.mult(h)
        
        point_r.soma(point_h)

        #Compara os pontos
        return point_s.eq(point_r)
    
    def ed2ec(self,x,y):
        if (x,y) == (0,1):
            return self.EC(0)
        
        z = (1 + y)/(1 - y) 
        w = z/x
        
        return self.EC(z/self.s + self.alfa , w/self.s)
    
    def ec2ed(self,P):
        if P == self.EC(0):
            return (0,1)
        
        x,y = P.xy()
        u = self.s * (x - self.alfa)
        v = self.s * y
        
        return (u / v, (u - 1) / (u + 1))

“edwards25519”

In [43]:
p = 2**255 - 19
b = 256
n = 254

K = GF(p)
a = K(-1)
d = K(-121665)/K(121666)
Px = K(15112221349535400772501151409588531511454012693041857206046113283949847762202)
Py = K(46316835694926478169428394003475163141307993866256225615783033603165251855960)

L = ZZ(2**252 + 27742317777372353535851937790883648493)
c = 3

In [44]:
#Gerar chave privada
private_key = os.urandom(32)

#Criar objeto
E = EdCDSA(p,b,n,K,a,d,Px,Py,L,c)
message = b"mensagem muito importante que vamos assinar"

#Calcular chave publica
public_key = E.public_key(private_key)
print("Public key: ", public_key)

#Assinar mensagem
signature = E.sign(message, private_key)
print("Signature: ", signature)

#Verificar que assinatura corresponde à mensagem
print("Verify: ", E.verify(message, signature, public_key))

Public key:  b'.\xeaR\xde\x11\xd3\xce\xcb\xdeRxd\x8e\x0cL3Le\x9c&\xb1\x82\x82P\xa4D\x8a\x01/\xfb\x85.'
Signature:  b'\xb8\xb3\x1ay\xd8\x0f\xb2{\xfb\xd4\x9a}6^PZW\xba\xaeA\xf7Qv\x14\xaf\xfd\x0e\xb5\x8f\x0c\xe7A\x8a\r\xa4\x97\x06k\xf2\xeb\xcf\xa8\xbe\x1f\xa5\x92\xff\x1b\x9e\x84\x83\x96E\xd8\r>\x7f\xcbXY\x03N8\r'


TypeError: unsupported operand type(s) for &: 'sage.rings.finite_rings.integer_mod.IntegerMod_gmp' and 'sage.rings.finite_rings.integer_mod.IntegerMod_gmp'

In [ ]:
#Gerar chave privada
private_key = os.urandom(32)

#Criar objeto
E = EdCDSA(p,b,n,K,a,d,Px,Py,L,c)
message = b"mensagem muito importante que vamos assinar"

#Calcular chave publica
public_key = E.public_key(private_key)
print("Public key: ", public_key)

#Assinar mensagem
signature = E.sign(message, private_key)
print("Signature: ", signature)

#Modificar mensagem
message = b"mensagem muito importante que iriamos assinar"

#Verificar que assinatura corresponde à mensagem
print("Verify: ", E.verify(message, signature, public_key))

In [ ]:
E_A = EdCDSA(p,b,n,K,a,d,Px,Py,L,c)
E_B = EdCDSA(p,b,n,K,a,d,Px,Py,L,c)

private_key = os.urandom(32)


#ALICE desafia Bob a provar que conhece a sua chave
challenge_alice = os.urandom(32)

#BOB assina o desafio para provar que conhece a chave privada
public_key_bob = E.public_key(private_key)
signature_bob = E.sign(challenge_alice, private_key)
#BOB envia assinatura e chave publica para ALICE

#ALICE verifica o desafio
verify = E_A.verify(challenge_alice, signature_bob, public_key_bob)

#Se a verificação for verdadeira, então ALICE sabe que BOB conhece a chave privada
if(verify):
    print("Bob knows the private key")
else:
    print("Bob does not know the private key")

#### Testes